In [1]:
# Cell 1: Imports and Setup (Updated channel names)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

# Base paths configuration
base_paths = {
    "healthy": r"C:\NewHandPD\Healthy Signals\Signal",
    "patient": r"C:\NewHandPD\Patient Signals\Signal"
}

category_labels = {
    "healthy": 0,
    "patient": 1
}

# Updated channel names (excluding Microphone)
channel_names = [
    "Fingergrip", "Axial_Pressure",
    "Tilt_X", "Tilt_Y", "Tilt_Z"
]

In [2]:
# Cell 2: Enhanced File Parsing Function with Metadata Extraction
def parse_signal_file(file_path, label):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    meta_info = {}
    signal_start_idx = 0
    in_meta_section = False

    # Extract metadata
    for i, line in enumerate(lines):
        line = line.strip()
        
        if line == "#<meta>":
            in_meta_section = True
            continue
        elif line == "#</meta>":
            in_meta_section = False
            signal_start_idx = i + 1
            break
            
        if in_meta_section and line.startswith("#<") and line.endswith(">"):
            # Extract key-value pairs from metadata
            try:
                key = line[2:line.find(">")]  # Get text between #< and >
                value = line[line.find(">")+1:-1]  # Get text between > and <
                
                # Convert numeric values to appropriate type
                if value.isdigit():
                    value = int(value)
                elif value.replace('.', '', 1).isdigit():
                    value = float(value)
                elif value.lower() == 'true':
                    value = True
                elif value.lower() == 'false':
                    value = False
                elif value == '':
                    value = None
                    
                meta_info[key] = value
            except:
                continue

    # Load signal data and drop first channel (Microphone)
    signal_lines = lines[signal_start_idx:]
    signal_array = np.loadtxt(signal_lines, delimiter="\t")
    
    # Keep only channels 1-5 (drop channel 0 - Microphone)
    if signal_array.ndim == 1:
        signal_array = signal_array[1:6].reshape(1, -1)  # For single-row signals
    else:
        signal_array = signal_array[:, 1:6]  # For multi-row signals

    return signal_array, label, meta_info

In [3]:
# Cell 3: Main Processing (Now includes metadata in records)
all_records = []
sigMea_records = []
sigSp_records = []

for category, folder_path in base_paths.items():
    label = category_labels[category]
    print(f"Processing {category} files...")

    for file_name in tqdm(os.listdir(folder_path)):
        if not file_name.endswith(".txt"):
            continue
            
        file_path = os.path.join(folder_path, file_name)
        try:
            signal_data, label_val, meta_info = parse_signal_file(file_path, label)

            record = {
                "file_name": file_name,
                "label": label_val,
                "signal": signal_data,
                **meta_info  # Unpack all metadata into the record
            }

            all_records.append(record)
            
            # Filter for specific file prefixes
            if file_name.startswith("sigMea"):
                sigMea_records.append(record)
            elif file_name.startswith("sigSp"):
                sigSp_records.append(record)

        except Exception as e:
            print(f"Error parsing {file_name}: {e}")

Processing healthy files...


100%|██████████| 421/421 [00:04<00:00, 84.52it/s]


Processing patient files...


100%|██████████| 373/373 [00:08<00:00, 45.24it/s]


In [4]:
# Cell 4: Create DataFrames (Now includes metadata columns)
df_all = pd.DataFrame(all_records)
df_sigMea = pd.DataFrame(sigMea_records)
df_sigSp = pd.DataFrame(sigSp_records)

print(f"\nSummary:")
print(f"Total files processed: {len(all_records)}")
print(f"Files starting with 'sigMea': {len(sigMea_records)}")
print(f"Files starting with 'sigSp': {len(sigSp_records)}")

# Show metadata columns we've extracted
print("\nMetadata columns found:")
print([col for col in df_all.columns if col not in ['file_name', 'label', 'signal']])


Summary:
Total files processed: 792
Files starting with 'sigMea': 264
Files starting with 'sigSp': 264

Metadata columns found:
['Person_ID_Number', 'Age', 'Gender', 'Writing_Hand', 'Weight', 'Height', 'Smoker', 'Notice', 'Object', 'Object_Index', 'Pen', 'Samplerate', 'Time', 'Date', 'Comment', 'Surename', 'Forename']


In [5]:
# Cell 5: Combine sigMea and sigSp DataFrames
df_combined = pd.concat([df_sigMea, df_sigSp], ignore_index=True)
print(f"Combined DataFrame shape: {df_combined.shape}")

Combined DataFrame shape: (528, 20)


In [6]:
# Cell 5.5: Split data by Person_ID_Number
import numpy as np

# Extract unique Person IDs
person_ids = df_combined['Person_ID_Number'].unique()

# Random shuffle of Person IDs
np.random.seed(42)  # for reproducibility
np.random.shuffle(person_ids)

# Calculate split indices
n_ids = len(person_ids)
train_idx = int(0.7 * n_ids)
val_idx = int(0.85 * n_ids)

# Split Person IDs into train, validation, and test sets
train_ids = person_ids[:train_idx]
val_ids = person_ids[train_idx:val_idx]
test_ids = person_ids[val_idx:]

# Create DataFrames for each split
df_train = df_combined[df_combined['Person_ID_Number'].isin(train_ids)]
df_val = df_combined[df_combined['Person_ID_Number'].isin(val_ids)]
df_test = df_combined[df_combined['Person_ID_Number'].isin(test_ids)]

print("\nData Split Summary:")
print(f"Total number of unique persons: {n_ids}")
print(f"Training set: {len(train_ids)} persons, {len(df_train)} samples")
print(f"Validation set: {len(val_ids)} persons, {len(df_val)} samples")
print(f"Test set: {len(test_ids)} persons, {len(df_test)} samples")

# Verify no data leakage
train_ids_set = set(df_train['Person_ID_Number'].unique())
val_ids_set = set(df_val['Person_ID_Number'].unique())
test_ids_set = set(df_test['Person_ID_Number'].unique())

print("\nChecking for data leakage:")
print("Overlap between train and val:", len(train_ids_set & val_ids_set))
print("Overlap between train and test:", len(train_ids_set & test_ids_set))
print("Overlap between val and test:", len(val_ids_set & test_ids_set))


Data Split Summary:
Total number of unique persons: 61
Training set: 42 persons, 368 samples
Validation set: 9 persons, 80 samples
Test set: 10 persons, 80 samples

Checking for data leakage:
Overlap between train and val: 0
Overlap between train and test: 0
Overlap between val and test: 0


In [7]:
# Cell 6: Simplified Feature extraction helpers
def compute_basic_stats(signal):
    """Compute only basic statistics without entropy or zero crossings"""
    return {
        "mean": np.mean(signal),
        "std": np.std(signal),
        "min": np.min(signal),
        "max": np.max(signal),
        "range": np.max(signal) - np.min(signal),
        "median": np.median(signal),
    }

def compute_derivatives(signal, order=1, sampling_rate=1000):
    """Compute derivatives up to snap (2nd derivative)"""
    dt = 1 / sampling_rate
    derivative = signal.copy()
    for _ in range(order):
        derivative = np.gradient(derivative, dt)
    return derivative

def compute_mass(signal):
    """Compute motion mass (sum of absolute values)"""
    return np.sum(np.abs(signal))

def extract_features_from_window(window_data, sampling_rate=1000):
    """Simplified feature extraction without entropy or zero crossings"""
    features = {}
    
    for i, channel in enumerate(channel_names):
        signal = window_data[:, i]
        stats = compute_basic_stats(signal)
        for stat_name, stat_val in stats.items():
            features[f"{channel}_{stat_name}"] = stat_val

        # First and second derivatives
        jerk = compute_derivatives(signal, order=1, sampling_rate=sampling_rate)
        snap = compute_derivatives(signal, order=2, sampling_rate=sampling_rate)

        # Only mass features
        features[f"{channel}_jerk_mass"] = compute_mass(jerk)
        features[f"{channel}_snap_mass"] = compute_mass(snap)

    return features

In [8]:
# Cell 7: Enhanced Sliding window feature extraction with metadata
window_size = 1000  # 1 second
step_size = 500     # 50% overlap
sampling_rate = 1000

def extract_features_from_df(df):
    features = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        signal = row["signal"]
        label = row["label"]
        file_name = row["file_name"]
        
        # Get all metadata columns (excluding signal, label, and file_name)
        metadata = {k: v for k, v in row.items() 
                   if k not in ['signal', 'label', 'file_name']}

        if signal.shape[0] < window_size:
            continue  # Skip very short signals

        for start in range(0, signal.shape[0] - window_size + 1, step_size):
            end = start + window_size
            window_data = signal[start:end, :]

            window_features = extract_features_from_window(window_data, sampling_rate)
            window_features["label"] = label
            window_features["file_name"] = file_name
            window_features["start_index"] = start
            window_features["end_index"] = end
            
            # Add all metadata to the window features
            window_features.update(metadata)

            features.append(window_features)
    
    return pd.DataFrame(features)

# Extract features for each split
print("Extracting features for training set...")
df_features_train = extract_features_from_df(df_train)
print("Extracting features for validation set...")
df_features_val = extract_features_from_df(df_val)
print("Extracting features for test set...")
df_features_test = extract_features_from_df(df_test)

print("\nFeature extraction complete:")
print(f"Training set shape: {df_features_train.shape}")
print(f"Validation set shape: {df_features_val.shape}")
print(f"Test set shape: {df_features_test.shape}")

Extracting features for training set...


100%|██████████| 368/368 [00:09<00:00, 39.20it/s]


Extracting features for validation set...


100%|██████████| 80/80 [00:02<00:00, 27.59it/s]


Extracting features for test set...


100%|██████████| 80/80 [00:01<00:00, 48.94it/s]


Feature extraction complete:
Training set shape: (11425, 61)
Validation set shape: (3503, 61)
Test set shape: (1994, 61)


In [11]:
# Cell 8: Add delta features (Now preserves metadata columns)
def add_delta_features(df):
    delta_features = []
    grouped = df.groupby("file_name")

    # Define known metadata columns (these are columns we know are not feature columns)
    base_non_feature_cols = ['label', 'file_name', 'start_index', 'end_index']
    
    # Identify potential metadata columns (columns that are constant within each file)
    potential_metadata_cols = [
        col for col in df.columns 
        if col not in base_non_feature_cols 
        and df.groupby('file_name')[col].nunique().max() == 1
    ]
    
    # Combine base non-feature columns with metadata columns
    non_feature_cols = base_non_feature_cols + potential_metadata_cols

    for file_name, group in grouped:
        group = group.sort_values("start_index").reset_index(drop=True)
        group_delta = group.copy()

        # Identify feature columns (excluding non-feature columns)
        feature_cols = [col for col in group.columns if col not in non_feature_cols]
        
        # Compute deltas only for feature columns
        for col in feature_cols:
            group_delta[f"delta_{col}"] = group[col].diff()

        # Fill first row deltas with 0
        delta_cols = [f"delta_{col}" for col in feature_cols]
        group_delta.loc[0, delta_cols] = 0

        delta_features.append(group_delta)

    return pd.concat(delta_features, ignore_index=True)

# Process each split
print("Adding delta features to training set...")
df_delta_train = add_delta_features(df_features_train)
print("Adding delta features to validation set...")
df_delta_val = add_delta_features(df_features_val)
print("Adding delta features to test set...")
df_delta_test = add_delta_features(df_features_test)

print("\nFinal shapes:")
print(f"Training set: {df_delta_train.shape}")
print(f"Validation set: {df_delta_val.shape}")
print(f"Test set: {df_delta_test.shape}")

# Verify no NaNs in any split
print("\nChecking for NaNs:")
print("Training set NaNs:", df_delta_train.isna().any().any())
print("Validation set NaNs:", df_delta_val.isna().any().any())
print("Test set NaNs:", df_delta_test.isna().any().any())

Adding delta features to training set...
Adding delta features to validation set...
Adding delta features to validation set...
Adding delta features to test set...
Adding delta features to test set...

Final shapes:
Training set: (11425, 101)
Validation set: (3503, 103)
Test set: (1994, 103)

Checking for NaNs:
Training set NaNs: True
Validation set NaNs: True
Test set NaNs: True

Final shapes:
Training set: (11425, 101)
Validation set: (3503, 103)
Test set: (1994, 103)

Checking for NaNs:
Training set NaNs: True
Validation set NaNs: True
Test set NaNs: True
